<a href="https://colab.research.google.com/github/sravanreddy6745/Traffic-Sign-Classification/blob/master/optiizecnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import  matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle 
import random
import os

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
os.getcwd()

'/content'

In [5]:
with open('/content/drive/My Drive/Traffic-Sign-Classification/traffic-signs-data/train.p', 'rb') as f:
    train_data = pickle.load(f)
    
with open('/content/drive/My Drive/Traffic-Sign-Classification/traffic-signs-data/valid.p', 'rb') as f:
    val_data = pickle.load(f)
with open('/content/drive/My Drive/Traffic-Sign-Classification/traffic-signs-data/test.p', 'rb') as f:
    test_data = pickle.load(f)

In [6]:
X_train, y_train = train_data['features'], train_data['labels']
X_val, y_val = val_data['features'], val_data['labels']
X_test, y_test = test_data['features'], test_data['labels']

OptimizeCNNModel

In [7]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 3.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=5da734a7a4a51a06649c67800455e6f7becfed8e6e82e6b64cf546bb2c7bdcf3
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=f0bebe8cb47c9cc74405dca6b10019ee727ac969033bd71de7a93ed3430706f7
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [8]:
import tensorflow as tf
from tensorflow import keras

In [9]:
X_train=X_train/255.0
X_test=X_test/255.0

In [10]:
X_train[0].shape

(32, 32, 3)

In [11]:
X_train=X_train.reshape(len(X_train),32,32,3)
X_test=X_test.reshape(len(X_test),32,32,3)

In [12]:
def build_model(hp):
  model = keras.Sequential([
                            keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        padding='same',
        input_shape=(32,32,3)
        ),
        keras.layers.Conv2D(
            filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
            activation='relu',
            padding='same'
        ),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(
            filters=hp.Int('conv_3_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_3_kernel', values = [3,5]),
            activation='relu',
            padding='same'
        ),
        keras.layers.Conv2D(
            filters=hp.Int('conv_4_filter', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('conv_4_kernel', values = [3,5]),
            activation='relu',
            padding='same'
        ),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Dropout(rate=hp.Choice('drop',values=[0.2,0.3])),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units', min_value=124, max_value=1024, step=16),
            activation='relu'
        ),
        keras.layers.Dense(43, activation='softmax')
        ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model




In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [14]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=10,directory='output',project_name="TrafficSign")

In [15]:
tuner_search.search(X_train,y_train,epochs=5,validation_data=(X_val,y_val))

Epoch 1/5
1088/1088 [==============================] - 30s 27ms/step - loss: 0.9331 - accuracy: 0.7384 - val_loss: 45.2975 - val_accuracy: 0.8916
Epoch 2/5
1088/1088 [==============================] - 29s 27ms/step - loss: 0.0786 - accuracy: 0.9778 - val_loss: 29.4011 - val_accuracy: 0.9036
Epoch 3/5
1088/1088 [==============================] - 30s 27ms/step - loss: 0.0452 - accuracy: 0.9870 - val_loss: 25.6005 - val_accuracy: 0.9512
Epoch 4/5
1088/1088 [==============================] - 30s 27ms/step - loss: 0.0357 - accuracy: 0.9901 - val_loss: 29.4611 - val_accuracy: 0.9512
Epoch 5/5
1088/1088 [==============================] - 29s 27ms/step - loss: 0.0334 - accuracy: 0.9900 - val_loss: 25.3534 - val_accuracy: 0.9585


Epoch 1/5
1088/1088 [==============================] - 23s 21ms/step - loss: 3.7479 - accuracy: 0.0551 - val_loss: 3.5639 - val_accuracy: 0.0544
Epoch 2/5
1088/1088 [==============================] - 23s 21ms/step - loss: 3.4848 - accuracy: 0.0562 - val_loss: 3.5610 - val_accuracy: 0.0544
Epoch 3/5
1088/1088 [==============================] - 22s 21ms/step - loss: 3.4822 - accuracy: 0.0561 - val_loss: 3.5577 - val_accuracy: 0.0544
Epoch 4/5
1088/1088 [==============================] - 22s 21ms/step - loss: 3.4815 - accuracy: 0.0558 - val_loss: 3.5586 - val_accuracy: 0.0544
Epoch 5/5
1088/1088 [==============================] - 22s 21ms/step - loss: 3.4809 - accuracy: 0.0549 - val_loss: 3.5574 - val_accuracy: 0.0544


Epoch 1/5
1088/1088 [==============================] - 28s 26ms/step - loss: 3.5806 - accuracy: 0.0543 - val_loss: 3.5569 - val_accuracy: 0.0476
Epoch 2/5
1088/1088 [==============================] - 28s 26ms/step - loss: 3.4844 - accuracy: 0.0555 - val_loss: 3.5593 - val_accuracy: 0.0544
Epoch 3/5
1088/1088 [==============================] - 28s 26ms/step - loss: 3.4823 - accuracy: 0.0548 - val_loss: 3.5583 - val_accuracy: 0.0544
Epoch 4/5
1088/1088 [==============================] - 28s 25ms/step - loss: 3.4818 - accuracy: 0.0553 - val_loss: 3.5551 - val_accuracy: 0.0544
Epoch 5/5
1088/1088 [==============================] - 28s 26ms/step - loss: 3.4807 - accuracy: 0.0552 - val_loss: 3.5572 - val_accuracy: 0.0544


Epoch 1/5
1088/1088 [==============================] - 21s 20ms/step - loss: 0.6710 - accuracy: 0.8108 - val_loss: 25.2578 - val_accuracy: 0.9397
Epoch 2/5
1088/1088 [==============================] - 21s 19ms/step - loss: 0.0744 - accuracy: 0.9784 - val_loss: 14.3877 - val_accuracy: 0.9528
Epoch 3/5
1088/1088 [==============================] - 21s 19ms/step - loss: 0.0467 - accuracy: 0.9855 - val_loss: 33.8748 - val_accuracy: 0.9143
Epoch 4/5
1088/1088 [==============================] - 21s 19ms/step - loss: 0.0387 - accuracy: 0.9892 - val_loss: 30.3412 - val_accuracy: 0.9299
Epoch 5/5
1088/1088 [==============================] - 21s 19ms/step - loss: 0.0329 - accuracy: 0.9901 - val_loss: 25.2842 - val_accuracy: 0.9510


Epoch 1/5
1088/1088 [==============================] - 28s 26ms/step - loss: 3.9091 - accuracy: 0.0562 - val_loss: 3.5574 - val_accuracy: 0.0544
Epoch 2/5
1088/1088 [==============================] - 28s 26ms/step - loss: 3.4844 - accuracy: 0.0565 - val_loss: 3.5601 - val_accuracy: 0.0476
Epoch 3/5
1088/1088 [==============================] - 28s 26ms/step - loss: 3.4829 - accuracy: 0.0559 - val_loss: 3.5607 - val_accuracy: 0.0476
Epoch 4/5
1088/1088 [==============================] - 28s 26ms/step - loss: 3.4811 - accuracy: 0.0546 - val_loss: 3.5646 - val_accuracy: 0.0544
Epoch 5/5
1088/1088 [==============================] - 28s 26ms/step - loss: 3.4805 - accuracy: 0.0554 - val_loss: 3.5585 - val_accuracy: 0.0544


...

Epoch 1/5
1088/1088 [==============================] - 22s 21ms/step - loss: 4.1839 - accuracy: 0.0543 - val_loss: 3.5670 - val_accuracy: 0.0544
Epoch 2/5
1088/1088 [==============================] - 22s 20ms/step - loss: 3.4842 - accuracy: 0.0560 - val_loss: 3.5662 - val_accuracy: 0.0544
Epoch 3/5
1088/1088 [==============================] - 22s 20ms/step - loss: 3.4828 - accuracy: 0.0557 - val_loss: 3.5573 - val_accuracy: 0.0544
Epoch 4/5
1088/1088 [==============================] - 22s 21ms/step - loss: 3.4812 - accuracy: 0.0553 - val_loss: 3.5557 - val_accuracy: 0.0544
Epoch 5/5
1088/1088 [==============================] - 22s 20ms/step - loss: 3.4812 - accuracy: 0.0552 - val_loss: 3.5543 - val_accuracy: 0.0476


Epoch 1/5
1088/1088 [==============================] - 24s 22ms/step - loss: 3.5870 - accuracy: 0.0545 - val_loss: 3.5662 - val_accuracy: 0.0544
Epoch 2/5
1088/1088 [==============================] - 23s 21ms/step - loss: 3.4845 - accuracy: 0.0572 - val_loss: 3.5571 - val_accuracy: 0.0544
Epoch 3/5
1088/1088 [==============================] - 24s 22ms/step - loss: 3.4829 - accuracy: 0.0545 - val_loss: 3.5555 - val_accuracy: 0.0476
Epoch 4/5
1088/1088 [==============================] - 23s 21ms/step - loss: 3.4810 - accuracy: 0.0552 - val_loss: 3.5555 - val_accuracy: 0.0476
Epoch 5/5
1088/1088 [==============================] - 23s 21ms/step - loss: 3.4804 - accuracy: 0.0566 - val_loss: 3.5562 - val_accuracy: 0.0544


Epoch 1/5
1088/1088 [==============================] - 22s 21ms/step - loss: 0.8835 - accuracy: 0.7466 - val_loss: 40.0949 - val_accuracy: 0.9098
Epoch 2/5
1088/1088 [==============================] - 22s 20ms/step - loss: 0.0909 - accuracy: 0.9734 - val_loss: 67.9336 - val_accuracy: 0.8925
Epoch 3/5
1088/1088 [==============================] - 22s 20ms/step - loss: 0.0643 - accuracy: 0.9813 - val_loss: 49.6120 - val_accuracy: 0.8628
Epoch 4/5
1088/1088 [==============================] - 22s 20ms/step - loss: 0.0534 - accuracy: 0.9843 - val_loss: 55.8372 - val_accuracy: 0.9041
Epoch 5/5
1088/1088 [==============================] - 22s 20ms/step - loss: 0.0414 - accuracy: 0.9882 - val_loss: 52.7210 - val_accuracy: 0.9172


Epoch 1/5
1088/1088 [==============================] - 24s 22ms/step - loss: 3.8147 - accuracy: 0.0556 - val_loss: 3.6278 - val_accuracy: 0.0553
Epoch 2/5
1088/1088 [==============================] - 23s 21ms/step - loss: 3.4850 - accuracy: 0.0551 - val_loss: 3.6370 - val_accuracy: 0.0510
Epoch 3/5
1088/1088 [==============================] - 24s 22ms/step - loss: 3.4828 - accuracy: 0.0546 - val_loss: 3.6376 - val_accuracy: 0.0544
Epoch 4/5
1088/1088 [==============================] - 23s 21ms/step - loss: 3.4815 - accuracy: 0.0549 - val_loss: 3.6528 - val_accuracy: 0.0531
Epoch 5/5
1088/1088 [==============================] - 23s 21ms/step - loss: 3.4808 - accuracy: 0.0560 - val_loss: 3.6348 - val_accuracy: 0.0524


INFO:tensorflow:Oracle triggered exit


In [16]:
model=tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 96)        7296      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 112)       268912    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 112)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        64576     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 8, 8, 64)          0

In [18]:
score=model.evaluate(X_test,y_test)

395/395 [==============================] - 4s 10ms/step - loss: 0.3087 - accuracy: 0.9517


In [19]:
score


[0.30869564414024353, 0.9517022967338562]

In [20]:
#model.save('/content/drive/My Drive/cnnoptiize')